In [1]:
from hypergraphs import *
from kegg import *

In [2]:
superpathwayDict = {
    "Carbohydrate metabolism":
    [
        ("00010", "Glycolysis / Gluconeogenesis"),
        ("00020", "Citrate cycle (TCA cycle)"),
        ("00030", "Pentose phosphate pathway"),
        ("00040", "Pentose and glucuronate interconversions"),
        ("00051", "Fructose and mannose metabolism"),
        ("00052", "Galactose metabolism"),
        ("00053", "Ascorbate and aldarate metabolism"),
        ("00500", "Starch and sucrose metabolism"),
        ("00520", "Amino sugar and nucleotide sugar metabolism"),
        ("00620", "Pyruvate metabolism"),
        ("00630", "Glyoxylate and dicarboxylate metabolism"),
        ("00640", "Propanoate metabolism"),
        ("00650", "Butanoate metabolism"),
        ("00562", "Inositol phosphate metabolism")
    ]
}
organismDict = {"hsa": "Homo sapiens (human)"}
outputDirectory = "/Users/boldi/Desktop/pw/"
dataDirectory = "../../LaTeX/Data/KEGG-Pathways/hsa"

In [3]:
organism = "hsa"
superpathway = "Carbohydrate metabolism"

In [4]:
safeSuperpathway = makesafe(superpathway)

In [5]:
H, sr2r, r2ss, rid2rname, mid2mname = read_hg_from_SBML([os.path.join(f"{dataDirectory}", f"{organism}{x[0]}-sbml.xml") for x in superpathwayDict[superpathway]])

/Users/boldi/.local/share/virtualenvs/qf-BIT_cIX4/lib/python3.8/site-packages/hypernetx/classes/hypergraph.py:968: UserWarning: Cannot add edge. Edge already in hypergraph
  warnings.warn("Cannot add edge. Edge already in hypergraph")


In [6]:
dd=hyper_cardon_crochemore(H, proxy=sr2r)

In [7]:
mapKeys(dd, onlyNonTrivial=True, prefOnly="r", keyfamily=r2ss)

[{'rnR04432', 'rnR12356'},
 {'rnR01648', 'rnR10183'},
 {'rnR01184', 'rnR01802'},
 {'rnR02527', 'rnR02530'},
 {'rnR00709', 'rnR01899'},
 {'rnR00227', 'rnR00742'},
 {'rnR01049', 'rnR01051', 'rnR01057'},
 {'rnR03363', 'rnR05802'},
 {'rnR00768', 'rnR04779'},
 {'rnR01431', 'rnR01904'},
 {'rnR03393', 'rnR05801'},
 {'rnR03430', 'rnR03434'},
 {'rnR03478', 'rnR04372'},
 {'rnR00405', 'rnR00432'},
 {'rnR00214', 'rnR00216'},
 {'rnR03427', 'rnR03428', 'rnR03429'},
 {'rnR00908', 'rnR01000', 'rnR02921', 'rnR02957', 'rnR04215'},
 {'rnR10928', 'rnR12124'},
 {'rnR02640', 'rnR02933'},
 {'rnR01790', 'rnR02111'},
 {'rnR03394', 'rnR03433', 'rnR05800'},
 {'rnR04513', 'rnR09827'},
 {'rnR03469', 'rnR05795', 'rnR11680'},
 {'rnR02749', 'rnR02750'},
 {'rnR00945', 'rnR01221', 'rnR01392', 'rnR01394'},
 {'rnR00022', 'rnR00306', 'rnR01103', 'rnR01175', 'rnR01185', 'rnR01737'},
 {'rnR00009',
  'rnR00010',
  'rnR00100',
  'rnR00253',
  'rnR00261',
  'rnR00317',
  'rnR00935',
  'rnR01176',
  'rnR01206',
  'rnR01334',
  

In [8]:
mapKeys(dd, onlyNonTrivial=True, keydesc=mid2mname, prefOnly="C")

[{('C05400', 'C05400'), ('C05401', 'C05401')},
 {('C00334', 'C00334'), ('C18060', 'C18060')},
 {('C00026', 'C00026'), ('C05379', 'C05379')},
 {('C00672', 'C00672'), ('C01801', 'C01801')},
 {('C00119', 'C00119'), ('C00121', 'C00121'), ('C00620', 'C00620')},
 {('C00037', 'C00037'), ('C00168', 'C00168')},
 {('C01204', 'C01204'), ('C04520', 'C04520')},
 {('C00181', 'C00181'), ('C00312', 'C00312')},
 {('C00937', 'C00937'), ('C03451', 'C03451')},
 {('C00203', 'C00203'), ('C00232', 'C00232')},
 {('C04063', 'C04063'), ('C04477', 'C04477')},
 {('C00099', 'C00099'),
  ('C00109', 'C00109'),
  ('C00222', 'C00222'),
  ('C00789', 'C00789'),
  ('C00818', 'C00818'),
  ('C01068', 'C01068'),
  ('C02670', 'C02670'),
  ('C03410', 'C03410'),
  ('C03691', 'C03691'),
  ('C05984', 'C05984')},
 {('C00618', 'C00618'), ('C01040', 'C01040')},
 {('C01220', 'C01220'), ('C11555', 'C11555')},
 {('C00014', 'C00014'),
  ('C00065', 'C00065'),
  ('C00258', 'C00258'),
  ('C01146', 'C01146')},
 {('C00007', 'C00007'),
  ('C

In [9]:
classSizeDist(dd, prefOnly="r")

{1: 132, 2: 31, 3: 6, 5: 1, 4: 1, 6: 1, 25: 1}

In [10]:
classSizeDist(dd, prefOnly="C")

{1: 126, 2: 15, 3: 1, 10: 1, 4: 1, 14: 1, 26: 1}

In [11]:
index = []

In [12]:
writeKEGGHtml(dd, os.path.join(outputDirectory, organism, safeSuperpathway), safeSuperpathway + "R", 
              f"{organismDict[organism]}, {superpathway}, Reactions group",
              onlyNonTrivial=True, prefOnly="r", cutPref="rn")
index = [(f"{safeSuperpathway}R-cont-0.html", "Reaction groups")]

In [13]:
writeKEGGHtml(dd, os.path.join(outputDirectory, organism, safeSuperpathway), safeSuperpathway + "C", 
              f"{organismDict[organism]}, {superpathway}, Compound group",
              onlyNonTrivial=True, prefOnly="C", cutPref="")
index += [(f"{safeSuperpathway}C-cont-0.html", "Compound groups")]

In [14]:
groupedReactions = mapKeys(dd, prefOnly="rn")
r2gr = {}
r2grall = {}
for i, reactions in enumerate(groupedReactions):
    for reaction in reactions:
        r2gr[reaction] = i
        r2grall[reaction] = reactions

In [15]:
groupedCompounds = mapKeys(dd, prefOnly="C")
c2gc = {}
c2gcall = {}
for i, compounds in enumerate(groupedCompounds):
    for compound in compounds:
        c2gc[compound] = i + len(groupedReactions)
        c2gcall[compound] = compounds

In [16]:
index += writeKEGGpdf(outputDirectory, organism, superpathway, superpathwayDict, r2gr, r2grall, c2gc, c2gcall)

In [17]:
writeIndex(os.path.join(outputDirectory, organism, safeSuperpathway), index, 
           f"{organismDict[organism]}, {superpathway}")

In [18]:
with open(os.path.join(outputDirectory, organism, safeSuperpathway, "data.pkl"), "wb") as file:
    pickle.dump([dd, sr2r, r2ss, rid2rname, mid2mname], file)

In [19]:
writeIndex(os.path.join(outputDirectory, organism), 
           [("{}/index.html".format(makesafe(k)), k) for k in superpathwayDict.keys()],
           f"{organismDict[organism]}")

In [20]:
writeIndex(os.path.join(outputDirectory), 
          [(f"{k}/index.html", v) for k,v in organismDict.items()],
           "Pathways")

In [21]:
with open(os.path.join(outputDirectory, organism, safeSuperpathway, "data.pkl"), "rb") as file:
    dd, sr2r, r2ss, rid2rname, mid2mname = pickle.load(file)